In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import drive
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping 

drive.mount('/content/drive')
df1 = pd.read_csv('/content/drive/MyDrive/딥러닝/Colab Notebook/samsung.csv').drop('date', axis = 1)

'''
df1.dtypes # 문자형 없음
df1.isna().sum().sort_values(ascending = False).head(15) # 결측치 없음
# 상관관계 확인
df1_corr = df1.corr()
df1_corr_sort = df1_corr.sort_values('close', ascending = False)
df1_corr_sort['close'].head(10)

cols = ['close', 'start', 'high', 'low', 'volume', 'transactionPrice', 'capitalization']
sns.pairplot(data = df, vars = cols, hue = 'close', palette = 'husl')
plt.show()
'''
# 시간순으로 재정렬
df2 = df1.sort_index(ascending=False)
df3 = df2.reset_index(drop = True)

# 정규화
scaler1 = MinMaxScaler(feature_range = (0,1))
scale_cols = ['start', 'high', 'low', 'volume', 'transactionPrice', 'capitalization']
scaled1 = scaler1.fit_transform(df3[scale_cols])
df4 = pd.DataFrame(scaled1, columns = scale_cols)

scaler2 = MinMaxScaler(feature_range = (0,1))
scaled2 = scaler2.fit_transform(df3['close'].values.reshape(-1,1))
df5 = pd.DataFrame(scaled2, columns = ['close'])

cols_train = ['start', 'high', 'low', 'volume', 'transactionPrice', 'capitalization']
X_trainset = df4[cols_train]
y = df5

X_train, X_test, y_train, y_test = train_test_split(X_trainset, y, test_size = 0.2)

model = Sequential()
model.add(Dense(10, input_dim = 6, activation = 'relu', name = 'dense1'))
model.add(Dense(30, activation = 'relu', name = 'dense2'))
model.add(Dense(20, activation = 'relu', name = 'dense3'))
model.add(Dense(1, name = 'output'))
model.summary()

early_stopping_callback = EarlyStopping(monitor = 'val_loss', patience = 20)
modelpath = '/content/drive/MyDrive/딥러닝/모델저장용'
checkpointer = ModelCheckpoint(filepath = modelpath, monitor = 'val_loss', vervose = 0, save_best_only = True)

model.compile(loss = 'mse', optimizer = 'adam', metrics = ['mse'])
history = model.fit(X_train, y_train, epochs = 2000, batch_size = 30, validation_split = 0.25, verbose = 1, callbacks = [early_stopping_callback, checkpointer])

real_prices = []
pred_prices = []
X_num = []
n_iter = 0

Y_prediction = model.predict(X_test).flatten()
Y_pred_df = pd.DataFrame(Y_prediction)

for i in range(len(Y_prediction)):
  real = y_test.iloc[i]
  prediction = Y_pred_df.iloc[i]

  rescaled_actual = scaler2.inverse_transform(real.values.reshape(-1,1))
  rescaled_pred = scaler2.inverse_transform(prediction.values.reshape(-1,1))

  print('Real price: {}, Expected price: {}'.format(rescaled_actual,rescaled_pred))
  real_prices.append(rescaled_actual)
  pred_prices.append(rescaled_pred)
  n_iter = n_iter+1
  X_num.append(n_iter)

pred_prices2 = np.concatenate(pred_prices).tolist()
pred_prices3 = np.concatenate(pred_prices2).tolist()

real_prices2 = np.concatenate(real_prices).tolist()
real_prices3 = np.concatenate(real_prices2).tolist()

plt.plot(X_num, pred_prices3, label = 'Expected')
plt.plot(X_num, real_prices3, label = 'Real')
plt.legend(loc = 'upper right')
plt.show()



# 새 섹션